# **Age Prediction**

**Objective**: 
To predict a person's age using a CNN model that extracts facial features from images.

**Dataset**:
I will be using the [UTKFace](https://www.kaggle.com/datasets/jangedoo/utkface-new) dataset, which contains labeled facial images for age prediction.

**Approach**:
To tackle this problem, I have two options:
- Using a Pretrained Model (ResNet18):
    - I plan to reuse the ResNet18 architecture (a residual network with 18 layers, introduced in 2015). I will modify the final fully connected layer to suit a regression task, enabling the model to predict continuous age values.
- Building a Custom CNN from Scratch:
    - Alternatively, I may design and train a custom CNN tailored specifically for this task, focusing on optimizing the architecture for age prediction.

Thank you for your interest in my project!

<h2>Table of Contents</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="#1-import-data">1. Import Data</a></li>
    <li><a href="#2-building-and-training">2. Building And Training</a>
        <ul>
            <li><a href="#21-using-a-pretrained-model-resnet18">2.1 Using a Pretrained Model (ResNet18)</a></li>
            <li><a href="#22-building-a-custom-cnn-from-scratch">2.2 Building a Custom CNN from Scratch</a></li>
            <li><a href="#23-model-comparison">2.3 Model Comparison</a></li>
        </ul>
    </li>
</ul>
</div>

<hr>

In [ ]:
import pandas as pd 
import numpy as np 
import torch 
from torch import nn 

import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import torch.nn.functional as F

from torch.utils.data import DataLoader
import cv2

from PIL import Image
import os
import torchvision.models as models
import torch.optim as optim

from tqdm.autonotebook import tqdm

from torch.utils.tensorboard import SummaryWriter 

from torchvision.models import resnet18


## **1. Import Data**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jangedoo/utkface-new")

print("Path to dataset files:", path)

In [ ]:

class ReadDataset(Dataset):
    def __init__(self, root, train = True, transform=None):
        self.path = os.path.join(root, "UTKFace" if train else "crop_part1")
        self.images, self.labels = [], [] 
        
        # Define the default transform if none provided
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),  # c,w,h -> h,w,c
            transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Scale [-1, 1]
        ])
        
        # Read the image paths and labels
        for file_name in os.listdir(self.path):
            file_path = os.path.join(self.path, file_name)
            self.images.append(file_path)
            
            # Get age for label (assuming file_name format is like "age-gender-otherinfo.jpg")
            age = int(file_name.split("_")[0])
            self.labels.append(float(age))  # Label is the age
        

    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, index):
        # Load image using cv2 (in BGR format)
        img_path = self.images[index]
        img = cv2.imread(img_path)
        
        # Convert BGR to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Convert image to PIL Image to apply transforms
        img = Image.fromarray(img)
        
        if self.transform:
            img = self.transform(img)  # Apply transformations
        
        # Get the label as a tensor
        label = torch.tensor(self.labels[index], dtype=torch.float)
        
        return img, label

root = path
# get dataset
train_dataset = ReadDataset(root)
test_dataset = ReadDataset(root, train = False)
train_loader = DataLoader(train_dataset, batch_size= 64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size= 64, shuffle=True)


In [ ]:
train_dataset.__len__()

In [ ]:
image,label = train_dataset.__getitem__(10)
image.shape

In [ ]:
def show_batch(images, labels):
    index = int(np.random.random_integers(0, 64))
    # Un-normalize 
    images = images[index] * 0.5 + 0.5  #  [-1, 1] → [0, 1]

    # Hiển thị
    plt.figure(figsize=(15, 8))
    plt.imshow(images.permute(1, 2, 0))  # [C, H, W] → [H, W, C]
    if labels is not None:
        plt.title(f"Labels: {labels[index]}")
    plt.axis('off')
    plt.show()

In [ ]:

# get the first batch
images, labels = next(iter(train_loader))

# Hiển thị batch
show_batch(images,labels)


## **2. Building And Training**

### **2.1 Using a Pretrained Model (ResNet18)**

**Let's see ResNet18's Architecture**

In [ ]:
resnet_pretrain = resnet18(pretrained=True)
print(resnet_pretrain)

**Adapting this Model to be suitable of this Problem accordingly**

In [ ]:
class adapt_resnet18(nn.Module):
    def __init__(self):
        super().__init__()
        old_model = resnet18(pretrained=True)
        self.features =  nn.Sequential(
            old_model.conv1,
            old_model.bn1,
            old_model.relu,
            old_model.layer1,
            old_model.layer2,
            old_model.layer3,
            nn.AdaptiveAvgPool2d((1, 1)), 
        )
        
        # add Dropout Layers to avoid overfitting
        self.regression = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256,1)
        )
        
        
    def forward(self, input):
         x = self.features(input)
         x = torch.flatten(x, 1)  # flatten 
         x = self.regression(x)
         return x 


In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model_v1 = adapt_resnet18()

# specify optimizer
optimizer = optim.Adam(model_v1.parameters(), lr=1e-2, weight_decay=1e-5)

# loss function for regression
criterion = nn.MSELoss() 

In [ ]:
# create Directory to save model
if not os.path.exists('Models'):
    os.mkdir('Models')
    print("Directory 'Models' has been created.")
else:
    print("Directory 'Models' already exists.")

In [ ]:
# save best model including number of epoch, state_dict, optimizer and loss
def save_checkpoint(state, filename="checkpoint.pth.tar"):
    torch.save(state, filename)

In [ ]:
# saving the best model when training
def train_validate_and_save_best_mode(model, criterion, optimizer, epochs, file_name):

    train_loss_list, test_loss_list = [], []
    model.to(device)
    best_loss = float('inf')
    for epoch in range(1, epochs+1):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        train_loader_bar = tqdm(train_loader, desc = "Trainning", leave=False) # illustrate process
        
        for batch_idx, (data, target) in enumerate(train_loader_bar):
            # move tensors to GPU if CUDA is available
            data = data.to(device)
            target = target.to(device)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            target = target.float()
            target = target.view(-1, 1)
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)

            train_loader_bar.set_postfix(loss = loss.item())
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        
        test_loader_bar = tqdm(test_loader, desc = "Validate", leave=False)
        
        for batch_idx, (data, target) in enumerate(test_loader_bar):
            # move tensors to GPU if CUDA is available
            data = data.to(device)
            target = target.to(device)
            # forward pass: compute predicted outputs by passing inputs to the model

            with torch.no_grad():
                output = model(data)
                # calculate the batch loss
                target = target.float()
                target = target.view(-1, 1)
                loss = criterion(output, target)
                # update average validation loss 
                valid_loss += loss.item()*data.size(0)
                test_loader_bar.set_postfix(loss = valid_loss)
        
        # calculate average losses
        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(test_loader.sampler)

        if valid_loss < best_loss:
            best_loss = valid_loss
            save_checkpoint({
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': valid_loss
            }, filename=("Models/{}.pth.tar".format(file_name)))

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))
        
        train_loss_list.append(train_loss)
        test_loss_list.append(valid_loss)
    
    return train_loss_list, test_loss_list

In [ ]:
## visualization for training and validating 
def evaluate(nb_epoch, train_loss_list, test_loss_list):
    epoch_list = [i for i in range(1, nb_epoch+1)]
    plt.plot(epoch_list,train_loss_list, marker = "o" , color = "blue")
    plt.plot(epoch_list,test_loss_list, marker = "o" , color = "red")

    plt.show()
    

In [ ]:
## training
epochs = 20 
train_loss_list_v1, test_loss_list_v1 = train_validate_and_save_best_mode(model_v1,criterion, optimizer, epochs, "Using_Pretrained_Model")
evaluate(epochs,train_loss_list_v1,test_loss_list_v1)

### **2.2 Building a Custom CNN from Scratch**

In [ ]:
class AgeModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional layers
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # Output: (64, 55, 55)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), # (128, 54, 54)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # Output: (128, 27, 27)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), # (128,26,26)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # Output: (128, 13, 13)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=2), # 256, 14,14
            nn.ReLU(),
            nn.MaxPool2d(2, 2) # 256, 7,7
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1 ),# 6 * 6 * 256 
            nn.ReLU(),
        )

        # Fully connected layers
        self.fc1 = nn.Linear(12544, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)  # Output: age (regression)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)  # (3, 224, 224) → (64, 55, 55)
        x = self.conv2(x)  # (64, 55, 55) → (128, 27, 27)
        x = self.conv3(x)  # (128, 27, 27) → (128, 13, 13)
        x = self.conv4(x)  # → (256, 13, 13)
        x = self.conv5(x)  # → (256, 13, 13)

        x = x.view(x.size(0), -1)  # Flatten → (batch_size, 512*5*5)

        x = self.dropout(x) 
        x = F.relu(self.fc1(x))  # → (512)
        x = F.relu(self.fc2(x))  # → (128)
        x = self.fc3(x)  # → (1)

        return x

In [ ]:
model_v2 = AgeModel()

print(model_v2)

In [ ]:
# we have to initialize one more time because these keep track the parameter of the previous Model

# specify optimizer
optimizer_v2 = optim.Adam(model_v2.parameters(), lr=1e-2, weight_decay=1e-5)

# loss function for regression
criterion_v2 = nn.MSELoss() 

In [ ]:
## training
epochs = 20 
train_loss_list_v2, test_loss_list_v2 = train_validate_and_save_best_mode(model_v2,criterion_v2, optimizer_v2, epochs, "Custom_Model")
evaluate(epochs,train_loss_list_v2,test_loss_list_v2)

### **2.3 Model Comparison**

In [ ]:
def compare(train_loss_list_v1, test_loss_list_v1, train_loss_list_v2, test_loss_list_v2):
    
    epochs = range(1, len(train_loss_list_v1) + 1)

    plt.figure(figsize=(12, 5))

    # Plot training loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_loss_list_v1, label='Model 1 - Train Loss', marker='o')
    plt.plot(epochs, train_loss_list_v2, label='Model 2 - Train Loss', marker='x')
    plt.title('Training Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot test loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, test_loss_list_v1, label='Model 1 - Test Loss', marker='o')
    plt.plot(epochs, test_loss_list_v2, label='Model 2 - Test Loss', marker='x')
    plt.title('Test Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()
    
    
compare(train_loss_list_v1, test_loss_list_v1, train_loss_list_v2, test_loss_list_v2)

    